<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch 6. 여러 개의 값에 대한 조작

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### 6.1 문자열을 연결하는 쿼리
데이터를 다양한 관점으로 바라보기 위해 용도에 맞게 여러 개의 데이터를 연결해서 전처리해주는 경우가 많습니다. 샘플 데이터를 사용하여 주소를 연결하는 쿼리를 작성해보겠습니다. 

샘플 데이터를 생성합니다.

In [3]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.mst_user_location;
CREATE TABLE sqldata.mst_user_location(
    user_id   string
  , pref_name string
  , city_name string
);
INSERT INTO sqldata.mst_user_location
VALUES
    ('U001', '서울특별시', '강서구')
  , ('U002', '경기도수원시', '장안구'  )
  , ('U003', '제주특별자치도', '서귀포시')
;


""


In [4]:
%%bigquery --project mygcpproject-340112
SELECT 
  user_id
  , CONCAT(pref_name, city_name) AS pref_city
FROM
  sqldata.mst_user_location
;

,user_id,pref_city
0,U001,서울특별시강서구
1,U003,제주특별자치도서귀포시
2,U002,경기도수원시장안구


CONCAT 함수를 이용하여 문자열을 연결해주면 지정한 변수명인 pref_city에 연결된 주소가 출력되는 것을 확인할 수 있습니다. 

### 6.2 여러개의 값 비교하기

In [5]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.quarterly_sales;
CREATE TABLE sqldata.quarterly_sales (
    year integer
  , q1   integer
  , q2   integer
  , q3   integer
  , q4   integer
);

INSERT INTO sqldata.quarterly_sales
VALUES
    (2015, 82000, 83000, 78000, 83000)
  , (2016, 85000, 85000, 80000, 81000)
  , (2017, 92000, 81000, NULL , NULL )
;

""


우선 quarterly_sales라는 이름의 dataset을 만들어 줍니다. 

In [6]:
%%bigquery --project mygcpproject-340112
SELECT
    year
  , q1
  , q2
    -- Q1과 Q2 의 매출 변화 평가하기
  , CASE
    WHEN q1 < q2 THEN '+'
    WHEN q1 = q2 THEN '-'
    ELSE '-'
  END AS judge_q1_q2
    -- Q1과 Q2의 매출액 차이 계산하기
  , q2 - q1 AS diff_q2_q1
    -- Q1과 Q2의 매출변화를 1, 0, -1로 표현하기
  , SIGN(q2 - q1) AS sign_q2_q1
FROM
  sqldata.quarterly_sales
ORDER BY
  year
;

,year,q1,q2,judge_q1_q2,diff_q2_q1,sign_q2_q1
0,2015,82000,83000,+,1000,1
1,2016,85000,85000,-,0,0
2,2017,92000,81000,-,-11000,-1


quarterly_sales 데이터에서 year, q1, q2 칼럼을 가져온 뒤 CASE 함수를 이용하여 조건을 작성하였습니다. 조건에 따른 결과를 judge_q1_q2 로 지정한 뒤에 매출액의 차이와 매출 변화를 출력합니다. SIGN 은 매개변수가 양수일 때 1, 음수일 때 -1을 출력하는 함수입니다. 

### 6.3 연간 최대/최소 4분기 매출 찾기


In [7]:
%%bigquery --project mygcpproject-340112
SELECT
    year
    -- Q1~Q4의 최대 매출 구하기
  , greatest(q1, q2, q3, q4) AS greatest_sales
    -- Q1~Q4의 최소 매출 구하기
  , least(q1, q2, q3, q4) AS least_sales
FROM
  sqldata.quarterly_sales
ORDER BY
  year
;

,year,greatest_sales,least_sales
0,2015,83000.0,78000.0
1,2016,85000.0,80000.0
2,2017,NaN,NaN


## 6.4 연간 평균 4분기 매출 구하기
greatest 함수 또는 least 함수는 기본 제공되는 함수이기 때문에 이를 활용하여 여러 개의 컬럼을 처리하는 경우를 다뤄보겠습니다. 

In [8]:
%%bigquery --project mygcpproject-340112
SELECT
    year
  , (q1 + q2 + q3 + q4) / 4 AS average
FROM
  sqldata.quarterly_sales
ORDER BY
  year
;

,year,average
0,2015,81500.0
1,2016,82750.0
2,2017,NaN


average 함수를 사용하여 분기 전체의 평균을 구하는 쿼리를 작성해보았습니다.  
SELECT 에서 year 과 분기의 평균을 average 변수로 지정했기 때문에 결과 테이블에 year 과 average 변수가 컬럼으로 출력된 것을 볼 수 있습니다.

## 6.5 NULL 값이 포함된 테이블 사칙연산하는 쿼리
NULL 값을 사칙연산하려면 COALESCE 함수를 사용하여 적절한 값으로 대체해야 합니다. 일단 2017년의 매출을 NULL 값을 0으로 변환하여 평균을 구해보도록 하겠습니다.

In [9]:
%%bigquery --project mygcpproject-340112
SELECT
    year
  , (COALESCE(q1, 0) + COALESCE(q2, 0) + COALESCE(q3, 0) + COALESCE(q4, 0)) / 4
    AS average
FROM
  sqldata.quarterly_sales
ORDER BY
  year
;

,year,average
0,2015,81500.0
1,2016,82750.0
2,2017,43250.0


다음과 같이 평균을 구하게 되면 NULL 값이 0으로 대체되어 평균을 구하기 때문에 결과값이 많이 낮아지게 됩니다. 2017년의 평균을 값이 있는 q1, q2값만으로 평균을 구하려면 값이 있는 컬럼의 수를 세서 나눠야 합니다. 컬럼의 수를 세서 나누는 쿼리를 작성하도록 하겠습니다.

## 6.6 NULL 이 아닌 컬럼만을 사용하여 평균값을 구하는 쿼리

In [10]:
%%bigquery --project mygcpproject-340112
SELECT
  year
  , (COALESCE(q1, 0) + COALESCE(q2, 0) + COALESCE(q3, 0) + COALESCE(q4, 0))
    / (SIGN(COALESCE(q1, 0)) + SIGN(coalesce(q2, 0))
      + SIGN(COALESCE(q3, 0)) + SIGN(COALESCE(q4, 0)))
  AS average
FROM
  sqldata.quarterly_sales
ORDER BY
  year
;

,year,average
0,2015,81500.0
1,2016,82750.0
2,2017,86500.0


6.5에서 나온 결과와 비교했을 때 NULL값을 포함하지 않고 평균을 구한 6.6 번의 테이블에서 2017년의 average가 다른 년도의 average값과 비슷하게 나온 것을 확인할 수 있습니다.

## 6.7 정수 자료형의 데이터 나누기
정수 자료형을 연산하기 위해 우선 데이터 테이블을 작성하겠습니다.

In [11]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.advertising_stats;
CREATE TABLE sqldata.advertising_stats(
    dt          string
  , ad_id       string
  , impressions integer
  , clicks      integer
);

INSERT INTO sqldata.advertising_stats
VALUES
    ('2017-04-01', '001', 100000,  3000)
  , ('2017-04-01', '002', 120000,  1200)
  , ('2017-04-01', '003', 500000, 10000)
  , ('2017-04-02', '001',      0,     0)
  , ('2017-04-02', '002', 130000,  1400)
  , ('2017-04-02', '003', 620000, 15000)
;


""


다음과 같이 데이터를 작성했습니다. 하루 데이터에서 각 광고의 클릭수를 노출수로 나눈 값을 계산해보겠습니다.

In [12]:
%%bigquery --project mygcpproject-340112
SELECT
  dt
,ad_id
  -- 정수를 나눌 때는 자동적으로 실수로 변환
  ,clicks/impressions AS ctr
  ,100.0*clicks/impressions AS ctr_as_percent
FROM
  sqldata.advertising_stats
WHERE
  dt='2017-04-01'
ORDER BY
  dt,ad_id
;

,dt,ad_id,ctr,ctr_as_percent
0,2017-04-01,001,0.03,3.0
1,2017-04-01,002,0.01,1.0
2,2017-04-01,003,0.02,2.0


여기서 2017-04-02 데이터는 impression이 0이기 때문에 0으로 나누게 되어 오류가 발생합니다. 0으로 나누는 것을 피하기 위해 CASE 식을 사용하여 impressions가 0인지 확인하면 됩니다. 

## 6.8 0으로 나누는 것을 피해 CTR을 계산하는 쿼리


In [13]:
%%bigquery --project mygcpproject-340112
SELECT
  dt
  ,ad_id
  -- CASE 식으로 분모가 0일 경우를 분기해서, 0으로 나누지 않게 하는 쿼리
  ,CASE
    WHEN impressions > 0 THEN 100.0 * clicks / impressions
  END AS ctr_as_percent_by_case
  --분모가 0이라면 NULL로 변환해서, 0으로 나누지 않게 만들기
  , 100.0*clicks/NULLIF(impressions, 0) AS ctr_as_percent_by_null
FROM
  sqldata.advertising_stats
ORDER BY
  dt, ad_id
;

,dt,ad_id,ctr_as_percent_by_case,ctr_as_percent_by_null
0,2017-04-01,001,3.000000,3.000000
1,2017-04-01,002,1.000000,1.000000
2,2017-04-01,003,2.000000,2.000000
3,2017-04-02,001,NaN,NaN
4,2017-04-02,002,1.076923,1.076923
5,2017-04-02,003,2.419355,2.419355


case 식을 이용하여 조건을 걸고, NULLIF를 이용해서 impressions값이 0이라면 NULL값을 출력하도록 하였습니다.

## 6.9 일차원 데이터의 절댓값과 제곱 평균 제곱근 계산하기
이번에는 입력한 두 값이 어느정도 떨어져있는지 나타내는 거리를 계산해보기 위해 데이터 테이블을 먼저 입력하겠습니다.

In [14]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.location_1d;
CREATE TABLE sqldata.location_1d(
    x1 integer
  , x2 integer
);

INSERT INTO sqldata.location_1d
VALUES
    ( 5 , 10)
  , (10 ,  5)
  , (-2 ,  4)
  , ( 3 ,  3)
  , ( 0 ,  1)
;

DROP TABLE IF EXISTS sqldata.location_2d;
CREATE TABLE sqldata.location_2d (
    x1 integer
  , y1 integer
  , x2 integer
  , y2 integer
);

INSERT INTO sqldata.location_2d
VALUES
    (0, 0, 2, 2)
  , (3, 5, 1, 2)
  , (5, 3, 2, 1)
;


""


location_1d 테이블에서 숫자 데이터를 기반으로 거리를 구해보도록 하겠습니다. 이 때 절댓값을 사용하는 방법, 제곱 평균 제곱근을 사용하는 방법 모두 사용해보겠습니다. 절대값을 사용할 때는 ABS함수, 제곱을 사용할 때는 POWER함수, 제곱근을 구할 때는 SQRT함수를 사용합니다.

In [15]:
%%bigquery --project mygcpproject-340112
SELECT
  abs(x1-x2) AS abs
  , sqrt(power(x1-x2, 2)) AS rms
FROM sqldata.location_1d
;

,abs,rms
0,6,6.0
1,1,1.0
2,0,0.0
3,5,5.0
4,5,5.0


다음과 같이 절댓값을 사용하여 구한 거리인 abs 값과 제곱하여 다시 제곱근을 구하는 방식으로 구한 rms 값을 볼 수 있습니다.

## 6.10 이차원 테이블에 대해 제곱 평균 제곱근 구하기
일차원 상의 거리를 구해보았습니다. 다음으로 이차원에서의 거리를 구해보도록 하겠습니다.

In [16]:
%%bigquery --project mygcpproject-340112
SELECT
  sqrt(power(x1-x2, 2) + power(y1-y2, 2)) AS dist
FROM sqldata.location_2d
;

,dist
0,3.605551
1,2.828427
2,3.605551


## 6.11 타임 스탬프를 활용하여 미래 또는 과거의 날짜/시간을 계산하기
기본적인 날짜/시간 데이터 계산 방법으로 정수의 덧셈과 뺄셈을 사용하는 방법을 알아보도록 하겠습니다. 

In [18]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.mst_users_with_dates;
CREATE TABLE sqldata.mst_users_with_dates (
    user_id        string
  , register_stamp string
  , birth_date     string
);

INSERT INTO sqldata.mst_users_with_dates
VALUES
    ('U001', '2016-02-28 10:00:00', '2000-02-29')
  , ('U002', '2016-02-29 10:00:00', '2000-02-29')
  , ('U003', '2016-03-01 10:00:00', '2000-02-29')
;


""


In [20]:
%%bigquery --project mygcpproject-340112
SELECT
    user_id
  , timestamp(register_stamp) AS register_stamp
  , timestamp_add(timestamp(register_stamp), interval 1 hour) AS after_1_hour
  , timestamp_sub(timestamp(register_stamp), interval 30 minute) AS before_30_minutes
  -- 타임스탬프 문자열을 기반으로 직접 날짜 계산을 할 수 없으므로 타임스탬프 자료형을 날짜/시간 자료형으로 변환한 뒤 계산하기
  , date(timestamp(register_stamp)) AS register_date
  , date_add(date(timestamp(register_stamp)), interval 1 day) AS after_1_day
  , date_sub(date(timestamp(register_stamp)), interval 1 month) AS before_1_month
FROM sqldata.mst_users_with_dates
;

,user_id,register_stamp,after_1_hour,before_30_minutes,register_date,after_1_day,before_1_month
0,U001,2016-02-28 10:00:00+00:00,2016-02-28 11:00:00+00:00,2016-02-28 09:30:00+00:00,2016-02-28,2016-02-29,2016-01-28
1,U002,2016-02-29 10:00:00+00:00,2016-02-29 11:00:00+00:00,2016-02-29 09:30:00+00:00,2016-02-29,2016-03-01,2016-01-29
2,U003,2016-03-01 10:00:00+00:00,2016-03-01 11:00:00+00:00,2016-03-01 09:30:00+00:00,2016-03-01,2016-03-02,2016-02-01


## 6.12 두 날짜의 차이 계산하기
방금 이용한 데이터를 사용하여 날짜간의 차이를 구해보도록 하겠습니다.

In [21]:
%%bigquery --project mygcpproject-340112
SELECT
    user_id
  , CURRENT_DATE AS today
  , date(timestamp(register_stamp)) AS register_date
  , date_diff(CURRENT_DATE, date(timestamp(register_stamp)), day) AS diff_days
FROM sqldata.mst_users_with_dates
;

,user_id,today,register_date,diff_days
0,U001,2022-02-11,2016-02-28,2175
1,U002,2022-02-11,2016-02-29,2174
2,U003,2022-02-11,2016-03-01,2173


## 6.14 연 부분 차이를 계산하는 쿼리

In [22]:
%%bigquery --project mygcpproject-340112
SELECT
  user_id
  , CURRENT_DATE AS today
  , date(timestamp(register_stamp)) AS register_date
  , date(timestamp(birth_date)) AS birth_date
  , date_diff(CURRENT_DATE, date(timestamp(birth_date)), year) AS current_age
  , date_diff(date(timestamp(register_stamp)), date(timestamp(birth_date)), year)
    AS register_age
FROM sqldata.mst_users_with_dates
;

,user_id,today,register_date,birth_date,current_age,register_age
0,U001,2022-02-11,2016-02-28,2000-02-29,22,16
1,U002,2022-02-11,2016-02-29,2000-02-29,22,16
2,U003,2022-02-11,2016-03-01,2000-02-29,22,16


이와 같이 bigquery에서 date_diff함수를 사용하여 day 단위가 아니라 year 단위로 출력하도록 할 수 있습니다. 그러나 연 부분의 차이만 계산될 뿐 해당 년의 생일이 넘었는지 계산되는 것이 아니기 때문에 제대로 된 나이가 계산되지 않습니다. 

## 6.15 날짜를 정수로 표현해서 나이를 계산하는 쿼리


In [23]:
%%bigquery --project mygcpproject-340112
-- 생일이 2000년 2월 29일인 사람의 2016년 2월 28일 시점의 나이 계산하기
SELECT floor((20160228-20000229) / 10000) AS age;

,age
0,15.0


## 6.16 등록 시점과 현재 시점의 나이를 문자열로 계산하기
다음 예는 샘플 데이터를 기반으로 등록시점과 현재 시점의 나이를 계산하는 쿼리를 작성해보겠습니다. 문자열에서 하이픈을 제거하고 정수로 cast하는 방식을 사용합니다. 

In [30]:
%%bigquery --project mygcpproject-340112
SELECT
    user_id
  , substring(register_stamp, 1, 10) AS register_date
  , birth_date
  -- 등록시점의 나이 계산하기
  , floor(
    (CAST(replace(substring(register_stamp, 1, 10), '-', '') AS int64)
    - CAST(replace(birth_date, '-', '') AS int64)
    ) / 10000
  ) AS register_age
  -- 현재 시점의 나이 계산하기
  , floor(
    (CAST(replace(CAST(CURRENT_DATE AS string), '-', '') AS int64)
    - CAST(replace(birth_date, '-', '') AS int64)
    )/10000
  )AS current_age
FROM sqldata.mst_users_with_dates
;

,user_id,register_date,birth_date,register_age,current_age
0,U001,2016-02-28,2000-02-29,15.0,21.0
1,U002,2016-02-29,2000-02-29,16.0,21.0
2,U003,2016-03-01,2000-02-29,16.0,21.0


## 6.19 IP 주소에서 4개의 10진수 부분을 추출하는 쿼리


In [31]:
%%bigquery --project mygcpproject-340112
SELECT
  ip
  , CAST(split(ip, '.')[SAFE_ORDINAL(1)] AS int64) AS ip_part_1
  , CAST(split(ip, '.')[SAFE_ORDINAL(2)] AS int64) AS ip_part_2
  , CAST(split(ip, '.')[SAFE_ORDINAL(3)] AS int64) AS ip_part_3
  , CAST(split(ip, '.')[SAFE_ORDINAL(4)] AS int64) AS ip_part_4
FROM
  (SELECT '192.168.0.1' AS ip) AS t
;

,ip,ip_part_1,ip_part_2,ip_part_3,ip_part_4
0,192.168.0.1,192,168,0,1


ip 부분에서 .을 기준으로 split 함수를 사용하여 ip를 나눈 결과가 출력되었습니다.